 # Modeling OpenFDA 2017 First Quarter Data of Critical Care Intravenous Medications [Ver. 0.1 08022017]

Author: by Brian O. Pauls (brianpauls@med.umich.edu)

 ### Describing the code and abbreviations from OpenFDA data
 
 The dataset from OpenFDA comes in the form of 7 separate ASCII text file delimited by '$'. 

File Descriptions for ASCII Data Files:

  1. DEMOyyQq.TXT contains patient demographic and administrative information,
a single record for each event report.
  2. DRUGyyQq.TXT contains drug/biologic information for as many medications as
were reported for the event (1 or more per event).
  3. REACyyQq.TXT contains all "Medical Dictionary for Regulatory Activities"
(MedDRA) terms coded for the adverse event (1 or more). For more information
on MedDRA, please contact the MSSO Help Desk at mssohelp@meddra.org. The
website is www.meddra.org.
  4. OUTCyyQq.TXT contains patient outcomes for the event (0 or more).
  5. RPSRyyQq.TXT contains report sources for the event (0 or more).
  6. THERyyQq.TXT contains drug therapy start dates and end dates for the
reported drugs (0 or more per drug per event)



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

import datetime as dt
%matplotlib inline

### Part I. Load 2017 Q1 FAERS data and display shortened views for quick referencing

In [2]:
### Define column names

demo_head = ['Primary_ID', 'Case_ID', 'Case_Version', 'Initial/Follow-up', 'AE_Start_dt', 'Mfr_Receive_AE_dt',
             'FDA_init_Receive_Case_dt', 'FDA_Receive_Case_dt', 'Report_Type', 'Reg_Auth_Case_num', 'mfr_Unique_Report_ID', 
             'mfr_sender_code', 'Lit_Reference', 'Age', 'Age_Unit', 'Pt_Age_Group','SEX', 'E-submission(Y/N)', 'Pt_Weight','Pt_weight_Unit', 
             'Report_Send_dt', 'Report_Send_to_mfr_dt', 'Reporter_Occupation', 'Reporter_country', 'Event_country']

indi_head = ['Primary_ID', 'Case_ID', 'Drug_Seq', 'MedDRA_indi_term']

outc_head = ['Primary_ID', 'Case_ID', 'Pt_Outcome']

reac_head = ['Primary_ID', 'Case_ID', 'MedDRA_reac_term', 'ReAdmin_Event_Data']

rpsr_head = ['Primary_ID', 'Case_ID', 'RpSr_Code']

ther_head = ['Primary_ID', 'Case_ID', 'Drug_Seq', 'Start_Dt', 'End_Dt', 'Therapy_Duration', 'Ther_Units']

drug_head = ['Primary_ID', 'Case_ID', 'Drug_Seq', 'Reporter_role', 'Drug_Name', 'Active_Ingredient', 'Value_VBM', 
             'Drug_Name_Source', 'Route', 'Verbatim_Dose' 'Cum_Dose_to_Rxn', 'Cum_Dose_to_Rxn_Units', 'Dechall_Code',
            'Rechall_Code','Lot_Numb', 'Drug_Exp_dt', 'NDA_Numn', 'Dose_Amount', 'Dose_Unit', 'Dose_Form', 'Dose_Freq' ]


In [3]:
#### load 2017 Q1 FAERS data from file

demographic_txt = pd.read_csv('faers_ascii_2017q1/ascii/DEMO17Q1.txt', delimiter="$",header = 0, names = demo_head, 
                              low_memory = False,skipinitialspace = True, parse_dates = [6,7])
indication_txt = pd.read_csv('faers_ascii_2017q1/ascii/INDI17Q1.txt', delimiter="$", header = 0, names = indi_head, 
                             low_memory = False, skipinitialspace = True)
outcome_txt = pd.read_csv('faers_ascii_2017q1/ascii/OUTC17Q1.txt', delimiter="$", header = 0, names = outc_head, 
                          low_memory = False, skipinitialspace = True)
reaction_txt = pd.read_csv('faers_ascii_2017q1/ascii/REAC17Q1.txt', delimiter="$", header = 0, names = reac_head, 
                           low_memory = False, skipinitialspace = True)
rptsource_txt = pd.read_csv('faers_ascii_2017q1/ascii/RPSR17Q1.txt', delimiter="$", header = 0, names = rpsr_head, 
                            low_memory = False, skipinitialspace = True)
therapy_txt = pd.read_csv('faers_ascii_2017q1/ascii/THER17Q1.txt', delimiter="$", header = 0, names = ther_head, 
                          low_memory = False, skipinitialspace = True)
drug_txt = pd.read_csv('faers_ascii_2017q1/ascii/DRUG17Q1.txt', delimiter="$", header = 0, names = drug_head, 
                       low_memory = False, skipinitialspace = False)


In [4]:
### Demographics dataframe
demographic_txt.reset_index(level = 0)
demographic_txt.fillna(value = 'Unknown' )
demographic_txt = pd.DataFrame(demographic_txt)

demographic_txt[:5] #--preview first 5 rows -

,Primary_ID,Case_ID,Case_Version,Initial/Follow-up,AE_Start_dt,Mfr_Receive_AE_dt,FDA_init_Receive_Case_dt,FDA_Receive_Case_dt,Report_Type,Reg_Auth_Case_num,...,Pt_Age_Group,SEX,E-submission(Y/N),Pt_Weight,Pt_weight_Unit,Report_Send_dt,Report_Send_to_mfr_dt,Reporter_Occupation,Reporter_country,Event_country
0,100039237,10003923,7,F,2012.0,20170111.0,2014-03-12,2017-01-19,EXP,NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
1,1000588614,10005886,14,F,20140116.0,20170126.0,2014-03-13,2017-02-01,EXP,NaN,...,NaN,M,Y,90.00,KG,20170201.0,NaN,MD,IL,IL
2,100063033,10006303,3,F,NaN,20140304.0,2014-03-13,2017-02-27,EXP,NaN,...,A,F,Y,NaN,NaN,20170227.0,NaN,OT,CA,CA
3,100065923,10006592,3,F,20170211.0,20170214.0,2014-03-13,2017-02-17,EXP,NaN,...,E,F,Y,NaN,NaN,20170217.0,NaN,OT,US,US
4,100085064,10008506,4,F,20140402.0,20170214.0,2014-03-13,2017-02-20,EXP,NaN,...,A,F,Y,NaN,NaN,20170220.0,NaN,OT,US,US


In [5]:
### Indications dataframe
indication_txt.reset_index(level = 0)
indication_txt.fillna(value = 'Unknown' )
indication_txt = pd.DataFrame(indication_txt)

indication_txt[:5] #--preview first 5 rows -

,Primary_ID,Case_ID,Drug_Seq,MedDRA_indi_term
0,100039237,10003923,1,Myelofibrosis
1,100039237,10003923,2,Polycythaemia vera
2,100039237,10003923,3,Malignant melanoma
3,100039237,10003923,4,Product used for unknown indication
4,100039237,10003923,5,Product used for unknown indication


In [6]:
### Outcomes dataframe
outcome_txt.reset_index(inplace = True)
outcome_txt.fillna(value = 'Unknown' )
outcome_txt = pd.DataFrame(outcome_txt)

outcome_txt[:5] #--preview first 5 rows -

,index,Primary_ID,Case_ID,Pt_Outcome
0,0,100039237,10003923,OT
1,1,1000588614,10005886,HO
2,2,100063033,10006303,HO
3,3,100063033,10006303,OT
4,4,100065923,10006592,DE


In [7]:
### Reaction dataframe
reaction_txt.reset_index(inplace = True)
reaction_txt.fillna(value = 'Unknown')
reaction_txt = pd.DataFrame(reaction_txt)

reaction_txt[:5] #--preview first 5 rows -

,index,Primary_ID,Case_ID,MedDRA_reac_term,ReAdmin_Event_Data
0,0,100039237,10003923,Blindness,NaN
1,1,100039237,10003923,Constipation,NaN
2,2,100039237,10003923,Diarrhoea,NaN
3,3,100039237,10003923,Dizziness,NaN
4,4,100039237,10003923,Dry eye,NaN


In [8]:
### Report Sources dataframe
rptsource_txt.reset_index(inplace = True, drop = True)
rptsource_txt.fillna (value = 'Unknown')
rptsource_txt = pd.DataFrame(rptsource_txt)

rptsource_txt[:5] #--preview first 5 rows -

,Primary_ID,Case_ID,RpSr_Code
0,129685711,12968571,HP
1,130157522,13015752,HP
2,130815331,13081533,CSM
3,130815371,13081537,CSM
4,130838641,13083864,HP


In [9]:
### Therapy dataframe
therapy_txt.reset_index(inplace = True)
therapy_txt.fillna(value = 'Unknown')
therapy_txt = pd.DataFrame(therapy_txt)

therapy_txt[:5] #--preview first 5 rows -

,index,Primary_ID,Case_ID,Drug_Seq,Start_Dt,End_Dt,Therapy_Duration,Ther_Units
0,0,100039237,10003923,1,2011.0,NaN,NaN,NaN
1,1,100039237,10003923,2,20120106.0,NaN,NaN,NaN
2,2,100039237,10003923,3,20160825.0,NaN,NaN,NaN
3,3,1000588614,10005886,1,201306.0,NaN,NaN,NaN
4,4,100063033,10006303,1,201203.0,NaN,NaN,NaN


In [10]:
### Drug_dataframe
drug_txt.reset_index(level = 0)
drug_txt.fillna(value = 'Unknown' )
drug_txt = pd.DataFrame(drug_txt)

### Remove cases where Active_Ingredient is not null
drug_txt1 = drug_txt[drug_txt['Active_Ingredient'].notnull()]
drug_txt1 = drug_txt1[drug_txt1['Drug_Name_Source'].notnull()]


## Test a query for all patients witha  report involving clonazepam
#drug_txt[drug_txt['Active_Ingredient'] == 'CLONAZEPAM']

In [ ]:
### template search for common ICU terms in drug_txt

med1_mdd = drug_txt1[drug_txt1.Active_Ingredient.str.contains('PIPERACILLIN')]
med1_mdd

### Part IV. Stratifying the 2017 QI FAERS cases based on Indications and Medications

#### B. Identifying indications of interest from 2017 Q1 FAERS data

### List of medications provided

 * PHENYLEPHERINE !GTT /
 * NOREPINEPHRINE EPINEPHRINE !GTT 
 * VASOPRESSIN !GTT
 * MILRINONE !GTT
 * AMIODARONE 
 * VECURONIUM !GTT
 * CISATRACURIUM 
 * PROPOFOL 
 * LORAZEPAM 
 * HYDROMORPHONE 
 * DEXMEDETOMIDINE 
 * BLOOD TRANSFUSION 
 * HEPARIN 
 * MEROPENEM  
 * VANCOMYCIN 
 * FLUCONAZOLE 
 * CASPOFUNGIN 
 * TOBRAMYCIN
 * PIPERACILLIN


In [97]:
### Find all patients in the first quarter 2017 data with any medication as mentioned above from medication list

### drop columns for variables which will not be utilized
drop_col_drug = ['Value_VBM','Drug_Name_Source','Route','Verbatim_DoseCum_Dose_to_Rxn',
                 'Dechall_Code','Rechall_Code','Lot_Numb', 'Drug_Exp_dt', 'NDA_Numn','Dose_Amount' , 
                 'Dose_Form', 'Dose_Freq', 'Drug_Name', 'Reporter_role', 'Primary_ID']

#all_key = ()
#case_key = drug_txt1[drug_txt1.columns[0:3]] ### Create a base key for all Case_IDs



In [41]:
### create a dataframe from drug_txt with only cases that contain drugs mentioned in emailed list

## PHENYLEPHERINE
drug_phenyl = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('PHENYLEPHRINE')])
drug_phenyl = drug_phenyl[drug_phenyl.Drug_Name_Source.str.contains('Intravenous')]
drug_phenylct = len(drug_phenyl)


In [42]:
## NOREPINEPHRINE 
drug_norepi = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('NOREPINEPHRINE')])
drug_norepi = drug_norepi[drug_norepi.Drug_Name_Source.str.contains('Intravenous')]
drug_norepict = len(drug_norepi)
all_drug = drug_phenyl.append(drug_norepi)


In [43]:
## EPINEPHRINE 
drug_epinep = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('EPINEPHRINE')])
#drug_epinep = drug_epinep[drug_epinep.Drug_Name_Source.str.contains('Intravenous')]
drug_epinepct = len(drug_epinep)
all_drug = all_drug.append(drug_epinep)


In [44]:
## VASOPRESSIN
drug_vasopres = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('VASOPRESSIN')])
#drug_vasopres = drug_vasopres[drug_vasopres.Drug_Name_Source.str.contains('Intravenous')]
drug_vasopresct = len(drug_vasopres)
all_drug = all_drug.append(drug_vasopres)


In [45]:
## MILRINONE
drug_milrin = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('MILRINONE')])
#drug_milrin = drug_milrin[drug_milrin.Drug_Name_Source.str.contains('Intravenous')]
drug_milrinct = len(drug_milrin)
all_drug = all_drug.append(drug_milrin)


In [46]:
## AMIODARONE
drug_amiod = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('AMIODARONE')])
drug_amiod = drug_amiod[drug_amiod.Drug_Name_Source.str.contains('Intravenous')]
drug_amiodct = len(drug_amiod)
all_drug = all_drug.append(drug_amiod)


In [47]:
## VECURONIUM GTT
drug_vecuro = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('VECURONIUM')])
drug_vecuro = drug_vecuro[drug_vecuro.Drug_Name_Source.str.contains('Intravenous')]
drug_vancoct = len(drug_vecuro)
all_drug = all_drug.append(drug_vecuro)


In [48]:
## CISATRACURIUM
drug_cisat = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('MILRINONE')])
drug_cisat = drug_cisat[drug_cisat.Drug_Name_Source.str.contains('Intravenous')]
drug_cisatct = len(drug_cisat)
all_drug = all_drug.append(drug_cisat)


In [49]:
## PROPOFOL
drug_propo = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('CISATRACURIUM')])
drug_propo = drug_propo[drug_propo.Drug_Name_Source.str.contains('Intravenous')]
drug_propoct = len(drug_propo)
all_drug = all_drug.append(drug_propo)


In [50]:
## LORAZEPAM
drug_loraz = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('LORAZEPAM')])
drug_loraz = drug_loraz[drug_loraz.Drug_Name_Source.str.contains('Intravenous')]
drug_lorazct = len(drug_loraz)
all_drug = all_drug.append(drug_loraz)


In [51]:
## HYDROMORPHONE
drug_hydromph = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('HYDROMORPHONE')])
drug_hydromph = drug_hydromph[drug_hydromph.Drug_Name_Source.str.contains('Intravenous')]
drug_hydromphct = len(drug_hydromph)
all_drug = all_drug.append(drug_hydromph)


In [52]:
## DEXMEDETOMIDINE
drug_dexmed = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('DEXMEDETOMIDINE')])
drug_dexmed = drug_dexmed[drug_dexmed.Drug_Name_Source.str.contains('Intravenous')]
drug_dexmedct = len(drug_dexmed)
all_drug = all_drug.append(drug_dexmed)


In [53]:
## BLOOD TRANSFUSION
drug_blood1 = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('BLOOD CELL')])
drug_blood1 = drug_blood1[drug_blood1.Drug_Name_Source.str.contains('Intravenous')]
#len(drug_blood1)

drug_blood2 = pd.DataFrame(drug_txt1[drug_txt1['Active_Ingredient'] == 'WHOLE BLOOD'])
#drug_blood2 = drug_blood2[drug_blood2.Drug_Name_Source.str.contains('Intravenous')]
#len(drug_blood2)

## merge the two kinds of blood transfusions
drug_bloodtrns = drug_blood1.append(drug_blood2)
drug_bloodtrnsct = len(drug_bloodtrns)
all_drug = all_drug.append(drug_bloodtrns)


In [54]:
## HEPARIN
drug_heparin = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('HEPARIN')])
drug_heparin = drug_heparin[drug_heparin.Drug_Name_Source.str.contains('Intravenous')]
drug_heparinct = len(drug_heparin)
all_drug = all_drug.append(drug_heparin)


In [55]:
## MEROPENEM
drug_meropen = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('MEROPENEM')])
#drug_meropen = drug_meropen[drug_meropen.Drug_Name_Source.str.contains('Intravenous')]
drug_meropenct = len(drug_meropen)
all_drug = all_drug.append(drug_meropen)


In [56]:
## VANCOMYCIN
drug_vanco = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('VANCOMYCIN')])
drug_vanco = drug_vanco[drug_vanco.Drug_Name_Source.str.contains('Intravenous')]
drug_vancoct = len(drug_vanco)
all_drug = all_drug.append(drug_vanco)
                 

In [57]:
## FLUCONAZOLE
drug_flucon = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('FLUCONAZOLE')])
drug_flucon = drug_flucon[drug_flucon.Drug_Name_Source.str.contains('Intravenous')]
drug_fluconct = len(drug_flucon)
all_drug = all_drug.append(drug_flucon)



In [58]:
## CASPOFUNGIN
drug_capso = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('CASPOFUNGIN')])
drug_capso = drug_capso[drug_capso.Drug_Name_Source.str.contains('Intravenous')]
drug_capsoct = len(drug_capso)
all_drug = all_drug.append(drug_capso)



In [59]:
## TOBRAMYCIN
drug_tobra = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('TOBRAMYCIN')])
drug_tobra = drug_tobra[drug_tobra.Drug_Name_Source.str.contains('Intravenous')]
drug_tobract = len(drug_tobra)
all_drug = all_drug.append(drug_tobra)


In [60]:
## PIPERACILLIN IV
drug_piper = pd.DataFrame(drug_txt1[drug_txt1.Active_Ingredient.str.contains('PIPERACILLIN')])
drug_piper = drug_piper[drug_piper.Drug_Name_Source.str.contains('Intravenous')]
drug_piperct = len(drug_piper)
all_drug = all_drug.append(drug_piper)


In [98]:
## Preview all medications dataframe
all_drug.drop(drop_col_drug, axis = 1, inplace = True)
all_drug[:10]


,Case_ID,Drug_Seq,Active_Ingredient,Cum_Dose_to_Rxn_Units,Dose_Unit
121591,12506465,30,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
169075,12746333,11,PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
245768,12961318,2,PHENYLEPHRINE HYDROCHLORIDE,NaN,UG/KG
245781,12961318,15,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,UG
247065,12963897,2,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
250095,12969366,10,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
253070,12974514,1,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
256900,12981946,5,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
257644,12983363,6,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN
260870,12990037,3,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE,NaN,NaN


In [99]:
### extract the case ID for cases identified above
case_key = all_drug[all_drug.columns[0:3]]

case_key[:5]

,Case_ID,Drug_Seq,Active_Ingredient
121591,12506465,30,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE
169075,12746333,11,PHENYLEPHRINE HYDROCHLORIDE
245768,12961318,2,PHENYLEPHRINE HYDROCHLORIDE
245781,12961318,15,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE
247065,12963897,2,PHENYLEPHRINE\PHENYLEPHRINE HYDROCHLORIDE


In [123]:
## create a dataframe for demographics of patients by case_key
demo_all = pd.merge(demographic_txt, case_key, on = ['Case_ID'], how = 'inner')

demo_all = demo_all[demo_all.SEX != 'UNK']

len(demo_all)
### save dataframe d to file
demo_all.to_csv('faers_ascii_2017q1/ascii/icuresult1.csv', index = False, header = True, sep = ',')

3334